# Imports

In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import os
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
import sklearn.preprocessing as sk
import random
from tensorflow.python.keras import backend as K
from sklearn.impute import SimpleImputer
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
GDSC_Expr = pd.read_csv("./Data New/GDSCv2.exprs.BDKANN.tsv", sep = "\t", index_col=0, decimal = ".")

CTRP_Expr = pd.read_csv("./Data New/CTRP.exprs.BDKANN.tsv", 
                    sep = "\t", index_col=0, decimal = ".")

#transpose so that input data is cell lines on rows and genes on columns 
GDSC_Expr = GDSC_Expr.T
CTRP_Expr = CTRP_Expr.T

GDSC_resp = pd.read_csv("./Data New/GDSCv2.aac.BDKANN.tsv",sep="\t")

CTRP_resp = pd.read_csv("./Data New/CTRP.aac.BDKANN.tsv",sep="\t")

GDSC_resp = GDSC_resp.T
CTRP_resp = CTRP_resp.T

# Match samples/indexes between expression and response
idx_GDSC = GDSC_Expr.index.intersection(GDSC_resp.index)
idx_CTRP = CTRP_Expr.index.intersection(CTRP_resp.index)
GDSC_Expr = GDSC_Expr.loc[idx_GDSC,:]
GDSC_resp = GDSC_resp.loc[idx_GDSC,:]
CTRP_Expr = CTRP_Expr.loc[idx_CTRP,:]
CTRP_resp = CTRP_resp.loc[idx_CTRP,:]

Mask1 = pd.read_csv("./Masks/M1New.csv", 
                    sep = ",", index_col=0, decimal = ".")

Mask2 = pd.read_csv("./Masks/M2New.csv", 
                    sep = ",", index_col=0, decimal = ".")

gene_names = pd.read_csv('./Masks/mart_export(3).txt',sep='\t')

gene_names = gene_names.drop_duplicates('NCBI gene (formerly Entrezgene) ID')

gene_dict = dict(zip(gene_names.iloc[:,1],gene_names.iloc[:,2]))

# Convert ensembl ids to entrez gene ids  and re-order to match mask 
GDSC_Expr = GDSC_Expr.loc[:,GDSC_Expr.columns.intersection(gene_names.iloc[:,1])]
GDSC_Expr = GDSC_Expr.rename(columns=gene_dict)
GDSC_Expr = GDSC_Expr.sort_index(axis=1)

CTRP_Expr = CTRP_Expr.loc[:,CTRP_Expr.columns.intersection(gene_names.iloc[:,1])]
CTRP_Expr = CTRP_Expr.rename(columns=gene_dict)
CTRP_Expr = CTRP_Expr.sort_index(axis=1)

Mask1 = Mask1.loc[CTRP_Expr.columns,:]

drugs = ['ATRA', 'Doxorubicin', 'Tamoxifen', 'Gefitinib', 'BIBW2992','masitinib', '17-AAG', 'GDC-0941', 'MK-2206', 'AZD6244', 'PLX4720']
drugIDs = ['15367', '28748', '41774', '49668', '61390', '63450', '64153', '65326', '67271','90227', '90295']
drug_dict = dict(zip(drugs,drugIDs))
GDSC_resp = GDSC_resp[GDSC_resp.columns.intersection(drugs)]
CTRP_resp = CTRP_resp[GDSC_resp.columns.intersection(drugs)]

CTRP_resp = CTRP_resp.rename(columns=drug_dict)
GDSC_resp = GDSC_resp.rename(columns=drug_dict)

imputer1 = SimpleImputer(missing_values=np.nan, strategy='mean')
GDSC_resp = imputer1.fit_transform(GDSC_resp.values)

imputer1 = SimpleImputer(missing_values=np.nan, strategy='mean')
CTRP_resp = imputer1.fit_transform(CTRP_resp.values)

lsP = Mask1.columns.intersection(Mask2.index)

Mask1 = Mask1.loc[:,lsP]
Mask2 = Mask2.loc[lsP,:]

Mask3 = pd.read_csv("./Masks/M3New.csv", 
                    sep = ",", index_col=0, decimal = ".")
lsD = Mask2.columns.intersection(Mask3.index)
Mask2 = Mask2.loc[:,lsD]
Mask3 = Mask3.loc[lsD,['49668', '67271', '90295', '41774']]

Mask1v2 = 1 - Mask1.values
Mask2v2 = 1 - Mask2.values
Mask3v2 = 1 - Mask3.values

In [3]:
#Custom layer for using masks
class CustomConnected(Dense):

    def __init__(self, units, Mask, **kwargs):
        #this is matrix A
        self.Mask = Mask                        
        #initalize the original Dense with all the usual arguments   
        super(CustomConnected,self).__init__(units,**kwargs)  
    def call(self, inputs):
        output = K.dot(inputs, self.kernel * self.Mask)
        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation is not None:
            output = self.activation(output)
        return output

In [4]:
# Regularization using masks for each layer
def l1_regL1(weight_matrix):
    return 0.0001 * K.sum(K.abs(weight_matrix*Mask1v2))
def l1_regL2(weight_matrix):
    return 0.0001 * K.sum(K.abs(weight_matrix*Mask2v2))
def l1_regL3(weight_matrix):
    return 0.0001 * K.sum(K.abs(weight_matrix*Mask3v2))

In [5]:
def create_FC_model(hidden1,hidden2,rate1,rate2):
    """Fully connected 2 hidden-layer model with optimized parameters"""
    model = Sequential()
    model.add(Dense(hidden1, input_dim=GDSC_Expr.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dropout(rate1))
    model.add(Dense(hidden2, activation='relu'))
    model.add(Dropout(rate2))
    model.add(Dense(GDSC_resp.shape[1], activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    return model

In [6]:
def create_FC_small_model(rate1,rate2):
    """create FC model with same arch as BDKANN"""
    model = Sequential()
    model.add(Dense(Mask2.shape[0], input_dim=GDSC_Expr.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dropout(rate1))
    model.add(Dense(Mask2.shape[1], activation='relu'))
    model.add(Dropout(rate2))
    model.add(Dense(GDSC_resp.shape[1], activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    return model

In [7]:
def create_BDKANN_model():
    """create BDKANN1 (only domain knowledge)"""
    model = Sequential()
    model.add(CustomConnected(Mask2.shape[0], Mask1v2, input_dim=GDSC_Expr.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(CustomConnected(Mask2.shape[1], Mask2v2, activation='relu'))
    model.add(CustomConnected(GDSC_resp.shape[1], Mask3v2, activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer='Adam', metrics=['mse','mae'])
    return model

In [8]:
def create_BDKANN2_model():
    """create BDKANN2 (with regularized extra edges)"""
    model = Sequential()
    model.add(Dense(Mask2.shape[0], input_dim=GDSC_expr.shape[1], kernel_initializer='random_normal', activation='relu', 
                    kernel_regularizer=l1_regL1))
    model.add(Dense(Mask2.shape[1], activation='relu', kernel_initializer='random_normal', kernel_regularizer=l1_regL2))
    model.add(Dense(GDSC_resp.shape[1], activation='linear', kernel_initializer='random_normal', kernel_regularizer=l1_regL3))
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    return model

In [9]:
param_dict = dict(batch_size=[16,32], 
                  #hidden1=[1024, 256, 128, 64, 16], 
                  #hidden2=[1024, 256, 128, 64, 16], 
                  rate1=[0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3], 
                  rate2=[0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3], 
                  epochs=[10, 20, 50, 100])

In [10]:
FC_model = keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_FC_model)
FC_small_model = keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_FC_small_model)

In [11]:
grid = RandomizedSearchCV(estimator=FC_small_model,param_distributions=param_dict,cv=3,n_iter=75)

In [12]:
grid.fit(CTRP_Expr,CTRP_resp,verbose=0)

scores = pd.DataFrame(grid.cv_results_)

scores.sort_values(by='mean_test_score',ascending=False,inplace=True)

scores.to_csv('FC_samearch_CV_scores.csv')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4448)              1961568   
_________________________________________________________________
dropout (Dropout)            (None, 4448)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 867)               3857283   
_________________________________________________________________
dropout_1 (Dropout)          (None, 867)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
7/7 [==============================] - 0s 2ms/step - loss: 0.0146 - mse: 0.0146 - mae: 0.0920
Model: "sequential

14/14 [==============================] - 0s 1ms/step - loss: 0.0059 - mse: 0.0059 - mae: 0.0540
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 4448)              1961568   
_________________________________________________________________
dropout_16 (Dropout)         (None, 4448)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 867)               3857283   
_________________________________________________________________
dropout_17 (Dropout)         (None, 867)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
14/14 [=======

14/14 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095 - mae: 0.0665
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 4448)              1961568   
_________________________________________________________________
dropout_32 (Dropout)         (None, 4448)              0         
_________________________________________________________________
dense_49 (Dense)             (None, 867)               3857283   
_________________________________________________________________
dropout_33 (Dropout)         (None, 867)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
14/14 [======

14/14 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082 - mae: 0.0575
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 4448)              1961568   
_________________________________________________________________
dropout_48 (Dropout)         (None, 4448)              0         
_________________________________________________________________
dense_73 (Dense)             (None, 867)               3857283   
_________________________________________________________________
dropout_49 (Dropout)         (None, 867)               0         
_________________________________________________________________
dense_74 (Dense)             (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
14/14 [======

7/7 [==============================] - 0s 2ms/step - loss: 0.0182 - mse: 0.0182 - mae: 0.1003
Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_96 (Dense)             (None, 4448)              1961568   
_________________________________________________________________
dropout_64 (Dropout)         (None, 4448)              0         
_________________________________________________________________
dense_97 (Dense)             (None, 867)               3857283   
_________________________________________________________________
dropout_65 (Dropout)         (None, 867)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
7/7 [==========

...

14/14 [==============================] - 0s 1ms/step - loss: 0.0078 - mse: 0.0078 - mae: 0.0562
Model: "sequential_192"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_576 (Dense)            (None, 4448)              1961568   
_________________________________________________________________
dropout_384 (Dropout)        (None, 4448)              0         
_________________________________________________________________
dense_577 (Dense)            (None, 867)               3857283   
_________________________________________________________________
dropout_385 (Dropout)        (None, 867)               0         
_________________________________________________________________
dense_578 (Dense)            (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
14/14 [=====

14/14 [==============================] - 0s 2ms/step - loss: 0.0399 - mse: 0.0399 - mae: 0.1659
Model: "sequential_200"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_600 (Dense)            (None, 4448)              1961568   
_________________________________________________________________
dropout_400 (Dropout)        (None, 4448)              0         
_________________________________________________________________
dense_601 (Dense)            (None, 867)               3857283   
_________________________________________________________________
dropout_401 (Dropout)        (None, 867)               0         
_________________________________________________________________
dense_602 (Dense)            (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
14/14 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.0114 - mse: 0.0114 - mae: 0.0726
Model: "sequential_208"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_624 (Dense)            (None, 4448)              1961568   
_________________________________________________________________
dropout_416 (Dropout)        (None, 4448)              0         
_________________________________________________________________
dense_625 (Dense)            (None, 867)               3857283   
_________________________________________________________________
dropout_417 (Dropout)        (None, 867)               0         
_________________________________________________________________
dense_626 (Dense)            (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
7/7 [=========

7/7 [==============================] - 0s 1ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0585
Model: "sequential_216"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_648 (Dense)            (None, 4448)              1961568   
_________________________________________________________________
dropout_432 (Dropout)        (None, 4448)              0         
_________________________________________________________________
dense_649 (Dense)            (None, 867)               3857283   
_________________________________________________________________
dropout_433 (Dropout)        (None, 867)               0         
_________________________________________________________________
dense_650 (Dense)            (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
7/7 [=========

7/7 [==============================] - 0s 2ms/step - loss: 0.0057 - mse: 0.0057 - mae: 0.0532
Model: "sequential_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_672 (Dense)            (None, 4448)              1961568   
_________________________________________________________________
dropout_448 (Dropout)        (None, 4448)              0         
_________________________________________________________________
dense_673 (Dense)            (None, 867)               3857283   
_________________________________________________________________
dropout_449 (Dropout)        (None, 867)               0         
_________________________________________________________________
dense_674 (Dense)            (None, 4)                 3472      
Total params: 5,822,323
Trainable params: 5,822,323
Non-trainable params: 0
_________________________________________________________________
7/7 [=========

In [14]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [13]:
scores

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rate2,param_rate1,param_epochs,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
59,8.035178,0.010195,0.143749,0.002387,0.01,0.01,100,16,"{'rate2': 0.01, 'rate1': 0.01, 'epochs': 100, ...",-0.007115,-0.005568,-0.007647,-0.006777,0.000882,1
74,4.660709,0.218353,0.136634,0.004102,0.05,0.05,100,32,"{'rate2': 0.05, 'rate1': 0.05, 'epochs': 100, ...",-0.006732,-0.005686,-0.007917,-0.006778,0.000912,2
8,8.075327,0.047134,0.140326,0.001049,0.01,0.2,100,16,"{'rate2': 0.01, 'rate1': 0.2, 'epochs': 100, '...",-0.007031,-0.005575,-0.007791,-0.006799,0.000919,3
4,4.470704,0.006798,0.129817,0.001820,0.05,0.2,100,32,"{'rate2': 0.05, 'rate1': 0.2, 'epochs': 100, '...",-0.007022,-0.005697,-0.008072,-0.006930,0.000971,4
11,8.037282,0.014212,0.139136,0.002796,0.2,0.2,100,16,"{'rate2': 0.2, 'rate1': 0.2, 'epochs': 100, 'b...",-0.007429,-0.005836,-0.007567,-0.006944,0.000786,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,1.235302,0.005960,0.142057,0.001331,0.1,0.3,10,16,"{'rate2': 0.1, 'rate1': 0.3, 'epochs': 10, 'ba...",-0.018792,-0.018351,-0.021155,-0.019432,0.001231,71
10,0.887074,0.016092,0.136476,0.002179,0.05,0.3,10,32,"{'rate2': 0.05, 'rate1': 0.3, 'epochs': 10, 'b...",-0.018708,-0.018198,-0.021539,-0.019482,0.001470,72
56,0.874085,0.004059,0.130260,0.001930,0.02,0.05,10,32,"{'rate2': 0.02, 'rate1': 0.05, 'epochs': 10, '...",-0.029117,-0.021293,-0.016526,-0.022312,0.005190,73
72,0.869889,0.006706,0.128243,0.002737,0.02,0.01,10,32,"{'rate2': 0.02, 'rate1': 0.01, 'epochs': 10, '...",-0.025378,-0.027208,-0.022109,-0.024898,0.002109,74
